# Modelagem

Nesta notebook é apresetado o processo de criação do modelo de classificação

## Leitura de pacote e bibliotecas necessários

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Leitura dos dados

In [2]:
dados = pd.read_csv('desafio_manutencao_preditiva_treino.csv')

## Preparação dos dados

Primeiramente são removidos os índices pois não têm poder preditivo.

In [3]:
dados.drop(['udi', 'product_id'], axis=1, inplace=True)

Como visto na análise exploratória as colunas `air_temperature` e `process_temperature_k` são fortemente correlacionadas. A distribuição de falhas também motrou-se semelhante entre as duas.

Pode-se então escolher apenas uma delas para simplificar o modelo. No contexto do projeto a variável `process_temperatre_k` parece ser mais relevante para a classificação da falha pois refere-se a temperatura do processo que da máquina.

In [4]:
dados = dados.drop(columns='air_temperature_k')

Duas outras variáveis altamente correlacionadas foram `torque_nm` e `rotational_spped_rpm`. Porém, a distribuição de falhas, para estas duas variáveis, é diferente. Por isso ambas serão mantidas na modelagem.

### Separação dos conjuntos de treino e teste.

Os dados serão separados randomicamente mas prcurando manter as mesmas proporções de tipo de falha e tipo de equipamento.

Proporção de tipos de equipamentos.

In [5]:
dados.type.value_counts(normalize=True)

L    0.603270
M    0.298035
H    0.098695
Name: type, dtype: float64

Proporções dos tipos de falhas.

In [6]:
dados.failure_type.value_counts(normalize=True)

No Failure                  0.965202
Heat Dissipation Failure    0.011249
Power Failure               0.009450
Overstrain Failure          0.007800
Tool Wear Failure           0.004500
Random Failures             0.001800
Name: failure_type, dtype: float64

Separação das variáveis independentes e dependente.

In [7]:
X = dados.drop(columns='failure_type') # Variáveis independentes
y = dados[['failure_type']] # Variável dependente

In [8]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=.3, stratify=y, random_state=101)


Proporção de tipos de equipamentos nos conjuntos de treino e teste.

In [9]:
X_treino.type.value_counts(normalize=True), X_teste.type.value_counts(normalize=True)

(L    0.607587
 M    0.297471
 H    0.094942
 Name: type, dtype: float64,
 L    0.593203
 M    0.299350
 H    0.107446
 Name: type, dtype: float64)

Poporções da falhas nos conjuntos de treino e test

In [10]:
y_treino.value_counts(normalize=True), y_teste.value_counts(normalize=True)

(failure_type            
 No Failure                  0.965281
 Heat Dissipation Failure    0.011359
 Power Failure               0.009430
 Overstrain Failure          0.007715
 Tool Wear Failure           0.004501
 Random Failures             0.001715
 dtype: float64,
 failure_type            
 No Failure                  0.965017
 Heat Dissipation Failure    0.010995
 Power Failure               0.009495
 Overstrain Failure          0.007996
 Tool Wear Failure           0.004498
 Random Failures             0.001999
 dtype: float64)

Portanto a separação utilizada conseguiu manter proporções semelhantes as do conjunto inicial.

### Separação das variáveis dummies.

Como os modelos performam melhor com valores numéricos será utilizado o método `get_dummies` do Pandas para transformar a coluna `type`, composta de valores categóricos, em colunas de valores binários (númericos).

In [11]:
X_treino = pd.get_dummies(X_treino)
X_teste = pd.get_dummies(X_teste)
X_treino.head()

,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,type_H,type_L,type_M
731,307.8,1392,43.9,0,0,0,1
3176,311.3,1409,51.8,16,0,1,0
5838,308.7,1352,53.0,171,0,0,1
2614,311.1,1338,53.3,63,0,0,1
5656,309.5,1520,38.1,106,0,1,0


### Padronização de variáveis.

Como as variáveis numéricas estão em diferentes escalas de grandeza será realizada a padronização dos valores para que fiquem em escalas de grandeza similares.

In [12]:
scaler = MinMaxScaler()
scaler.fit(X_treino)
columns = X_treino.columns
X_treino = scaler.transform(X_treino)
X_treino = pd.DataFrame(X_treino, columns=columns)
X_teste = scaler.transform(X_teste)
X_teste = pd.DataFrame(X_teste, columns=columns)
X_treino.describe()

,process_temperature_k,rotational_speed_rpm,torque_nm,tool_wear_min,type_H,type_L,type_M
count,4666.000000,4666.000000,4666.000000,4666.000000,4666.000000,4666.000000,4666.000000
mean,0.523390,0.214945,0.498706,0.438994,0.094942,0.607587,0.297471
std,0.186468,0.102709,0.137464,0.258372,0.293166,0.488340,0.457195
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.375000,0.147846,0.403846,0.215447,0.000000,0.000000,0.000000
50%,0.525000,0.195576,0.500000,0.439024,0.000000,1.000000,0.000000
75%,0.662500,0.258440,0.592033,0.658537,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Métricas de avaliação

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

Foram escolhidas 5 métricas de avaliação, utilizadas em modelos de classificação:
* Acurácia (`accuracy_score`): é a quantidade de acertos do modelo divido pelo total da amostra. O quanto o modelo acertou.
* Precisão (`precision_score`): avalia dentre todos os dados classificados como positivos, quantos são realmente positivos.
* Recall (`recall_score`): avalia a porcentagem de dados classificados como positivos comparado com a quantidade real de positivos que existem em nossa amostra.
* f1-score (`f1_score`): essa métrica une precisão e recall afim de trazer um número único que determine a qualidade geral do nosso modelo. Adequado para classes desbalanceadas.
* MCC score (`mathews_corrcoef`):  é uma métrica que mede a eficácia de um modelo de classificação, levando em conta a precisão, recall e F1-score.

In [14]:
# Dataframe para registrar as performances dos modelos
performaces = pd.DataFrame(columns=['Acurácia', 'Recall', 'Precisão', 'F1-Score',
                                     'MCC score', 'Tempo de treinamento', 
                                     'Tempo de predição', 'Tempo total'])

## Modelos

### Árvore de Decisão

Treinamento do modelo

In [15]:
%%time
from sklearn.tree import DecisionTreeClassifier
inicio = time.time()
model_dtc = DecisionTreeClassifier().fit(X_treino, y_treino)
fim_treino = time.time()
y_predito = model_dtc.predict(X_teste) # These are the predictions from the test data.
fim_previsao = time.time()

CPU times: user 169 ms, sys: 29.5 ms, total: 199 ms
Wall time: 198 ms


Avaliação do modelo

In [16]:
def avaliacao_modelo(y_teste, y_predito, modelo):
    acuracia = accuracy_score(y_teste, y_predito)
    precisao = precision_score(y_teste, y_predito, average='weighted')
    recall = recall_score(y_teste, y_predito, average='weighted')
    f1s = f1_score(y_teste, y_predito, average='weighted')
    MCC = matthews_corrcoef(y_teste, y_predito)

    print("Acurácia: "+ "{:.2%}".format(acuracia))
    print("Recall: "+ "{:.2%}".format(recall))
    print("Precisão: "+ "{:.2%}".format(precisao))
    print("F1-Score: "+ "{:.2%}".format(f1s))
    print("MCC: "+ "{:.2%}".format(MCC))
    print("Tempo de treino: "+ "{:.2f}".format(fim_treino - inicio)+" s")
    print("Tempo para previsão: "+"{:.2f}".format(fim_previsao - fim_treino)+" s")
    print("Total: "+"{:.2f}".format(fim_treino - inicio)+" s")
    performaces.loc[modelo] = [acuracia, recall, precisao, f1s,
                               MCC, fim_treino - inicio, 
                               fim_previsao - fim_treino,
                               fim_previsao - inicio]

avaliacao_modelo(y_teste, y_predito, 'Árvore de Decisão')

Acurácia: 95.80%
Recall: 95.80%
Precisão: 95.81%
F1-Score: 95.80%
MCC: 37.80%
Tempo de treino: 0.03 s
Tempo para previsão: 0.00 s
Total: 0.03 s


### Random Forest

In [17]:
%%time
from sklearn.ensemble import RandomForestClassifier
inicio = time.time()
model_rfc = RandomForestClassifier(n_estimators = 200,
                                   n_jobs=-1,
                                   random_state=101).fit(X_treino, y_treino.values.ravel())
fim_treino = time.time()
y_predito = model_rfc.predict(X_teste) 
fim_previsao = time.time()

CPU times: user 3.58 s, sys: 1.65 s, total: 5.23 s
Wall time: 2.52 s


In [18]:
avaliacao_modelo(y_teste, y_predito, 'Random Forest')

Acurácia: 97.50%
Recall: 97.50%
Precisão: 95.76%
F1-Score: 96.56%
MCC: 53.30%
Tempo de treino: 2.11 s
Tempo para previsão: 0.36 s
Total: 2.11 s


/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Gradient Boosting Classifier

In [19]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
inicio = time.time()
model_gbc = GradientBoostingClassifier().fit(X_treino, y_treino.values.ravel())
fim_treino = time.time()
y_predito = model_gbc.predict(X_teste)
fim_previsao = time.time()

CPU times: user 5.59 s, sys: 0 ns, total: 5.59 s
Wall time: 5.59 s


In [20]:
avaliacao_modelo(y_teste, y_predito, 'Gradient Boosting Classifier')

Acurácia: 97.20%
Recall: 97.20%
Precisão: 96.13%
F1-Score: 96.58%
MCC: 49.66%
Tempo de treino: 5.58 s
Tempo para previsão: 0.02 s
Total: 5.58 s


/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### XGBoost

Para o XGBoost é necessário que todas as variáveis sejam to tipo numéricas. Assim, será utilizado o `LabelEncoder` do Scikit-Learn para transformar os valores textuais de `y` para numéricos.

In [21]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_teste)

y_teste2 = encoder.transform(y_teste)
y_treino2 = encoder.transform(y_treino)

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [23]:
%%time
import xgboost as xgb

inicio = time.time()
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_treino, y_treino2)
fim_treino = time.time()
y_predito = model_xgb.predict(X_teste)
fim_previsao = time.time()

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype i

[10:35:10] WARNING: /croot/xgboost-split_1675119646044/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 10.6 s, sys: 8.96 ms, total: 10.6 s
Wall time: 1.5 s


In [24]:
avaliacao_modelo(y_teste2, y_predito, 'XGBoost Classifier')

Acurácia: 97.65%
Recall: 97.65%
Precisão: 96.91%
F1-Score: 97.04%
MCC: 58.53%
Tempo de treino: 1.32 s
Tempo para previsão: 0.01 s
Total: 1.32 s


/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


O comando a seguir exibe a tabela de resumo da avaliação dos modelos.

In [26]:
performaces.fillna(.90,inplace=True)
performaces.style.background_gradient(cmap='coolwarm').format({'Acurácia': '{:.2%}',
                                                               'Precisão': '{:.2%}',
                                                               'Recall': '{:.2%}',
                                                               'F1-Score': '{:.2%}',
                                                               'MCC score': '{:.2%}',
                                                               'Tempo de treinamento':'{:.1f}',
                                                               'Tempo de predição':'{:.1f}',
                                                               'Tempo total':'{:.1f}',
                                                               })

,Acurácia,Recall,Precisão,F1-Score,MCC score,Tempo de treinamento,Tempo de predição,Tempo total
Árvore de Decisão,95.80%,95.80%,95.81%,95.80%,37.80%,0.0,0.0,0.0
Random Forest,97.50%,97.50%,95.76%,96.56%,53.30%,2.1,0.4,2.5
Gradient Boosting Classifier,97.20%,97.20%,96.13%,96.58%,49.66%,5.6,0.0,5.6
XGBoost Classifier,97.65%,97.65%,96.91%,97.04%,58.53%,1.3,0.0,1.3


## Conclusões

Considerando as métricas apresentadas o XGBoost é a melhor solução para o problema apresentado. Na notebook `otimizacao_modelo.ipynb` será realizada a otimização dos parâmetros do XGBoost em busca de melhorar as previsões realizadas pelo mesmo.